# 6. RRR: Rewrite-Retrieve-Read

**What:** Rewrite vague/messy queries into clear, searchable queries

**Why:** Poor queries = poor retrieval results

**When:** Public chatbots, non-expert users, noisy input

**Core Idea:** Fix the query before searching, but answer the original question

**Examples:**
- "that molecule stuff" → "molecular optimization techniques"
- "the brain thing for AI" → "neural network architecture"

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from config import model, setup_vectorstore, get_retriever, format_docs

## Rewrite Query

In [ ]:
def rewrite_query(question):
    template = """Rewrite this vague or messy question into a clear, searchable query.
Use specific technical terms.

Original: {question}

Rewritten:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    return chain.invoke({"question": question}).strip()

# Test
vague_queries = [
    "that molecule stuff",
    "how do agents remember stuff",
    "What DMS?"
]

print("Query Rewrites:")
for q in vague_queries:
    print(f"  '{q}' → '{rewrite_query(q)}'")

## RRR Retrieval

In [ ]:
def rrr_retrieve(question, retriever, k=5):
    print(f"Original: {question}")
    
    rewritten = rewrite_query(question)
    print(f"Rewritten: {rewritten}\n")
    
    # Retrieve with rewritten query
    docs = retriever.invoke(rewritten)[:k]
    
    print(f"Retrieved {len(docs)} documents")
    for i, doc in enumerate(docs):
        source = doc.metadata.get('filename', 'unknown')
        print(f"  {i+1}. [{source}] {doc.page_content[:50]}...")
    
    return docs, rewritten

## Complete RRR RAG

In [ ]:
def rrr_rag(question, retriever, k=5):
    docs, rewritten = rrr_retrieve(question, retriever, k)
    context = format_docs(docs)
    
    # Answer the ORIGINAL question
    template = """Answer the ORIGINAL question based on context.

Context:
{context}

Original Question: {question}

Answer:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt | model | StrOutputParser()
    answer = chain.invoke({"context": context, "question": question})
    
    print(f"\nAnswer: {answer}")
    return answer

## Test

In [ ]:
vectorstore = setup_vectorstore()
retriever = get_retriever(vectorstore, k=5)

vague_queries = [
    "Where did Otabek study?",
    "What DMS?",
    "that molecule stuff"
]

for q in vague_queries:
    print("="*60)
    rrr_rag(q, retriever)
    print()